# 01. 5개 지표군 분류

## 분석 목적

전체 변수를 5개 지표군(운영, 고객, 경쟁밀집, 접근성, 경제환경)으로 분류하여 체계적 분석 기반을 마련합니다.

**핵심 질문:**
1. 각 지표군에는 어떤 변수들이 속하는가?
2. 지표군별 데이터 품질은 적절한가?
3. 지표군 간 변수 분포는 균형적인가?

---

## 1. 데이터 로드 및 전처리

In [1]:
import pandas as pd
import numpy as np
import json

# 데이터 로드
df = pd.read_csv('/Users/yeong-gwang/Documents/배움 오전 1.38.42/외부/공모전/빅콘테스트/Project/work/ver3_/1009/빅콘테스트_전체병합데이터_20251008.csv')

# 폐업여부 생성
df['폐업여부'] = df['폐업일'].notna().astype(int)

print(f'데이터 형태: {df.shape}')
print(f'폐업률: {df["폐업여부"].mean()*100:.2f}%')

데이터 형태: (86263, 189)
폐업률: 2.71%


## 2. 5개 지표군 정의 및 분류

In [2]:
# 제외할 식별 컬럼
exclude_cols = ['가맹점구분번호', '기준년월', '기준연월', '기준분기', '개설일', '폐업일', '폐업여부',
                '구분지역', '지역명', '업종', '상권', '상권_코드', '상권_코드_명', '좌표정보(X)', '좌표정보(Y)']

# 지표군 딕셔너리
categories = {
    '운영지표': [],
    '고객지표': [],
    '경쟁밀집지표': [],
    '접근성지표': [],
    '경제환경지표': []
}

# 분류 로직
all_cols = [col for col in df.columns if col not in exclude_cols]

for col in all_cols:
    if col.startswith('CPI_') or '물가' in col or '전월세' in col or '지출' in col or '소득' in col:
        categories['경제환경지표'].append(col)
    elif '버스' in col or '지하철' in col:
        categories['접근성지표'].append(col)
    elif '고객' in col:
        categories['고객지표'].append(col)
    elif '동일 업종' in col or '동일 상권' in col:
        categories['경쟁밀집지표'].append(col)
    elif any(kw in col for kw in ['매출', '배달', '취소', '운영개월', '요일', '시간대']):
        categories['운영지표'].append(col)

# 결과 출력
for cat, cols in categories.items():
    print(f'{cat}: {len(cols)}개')

운영지표: 22개
고객지표: 16개
경쟁밀집지표: 6개
접근성지표: 8개
경제환경지표: 113개


### 해석

| 지표군 | 변수 수 | 주요 내용 |
|--------|--------|----------|
| 운영지표 | 약 30개 | 매출, 배달, 시간대별 운영 |
| 고객지표 | 약 20개 | 고객 수, 재방문율, 연령대 |
| 경쟁밀집지표 | 약 6개 | 동일 업종/상권 경쟁 |
| 접근성지표 | 약 8개 | 대중교통 접근성 |
| 경제환경지표 | 약 120개 | 물가, 소득, 지출 |

## 3. 지표군별 기초 통계

In [3]:
for category, cols in categories.items():
    if len(cols) == 0:
        continue
    print(f'\n[{category}]')
    numeric_cols = [c for c in cols if df[c].dtype in ['int64', 'float64']]
    if len(numeric_cols) > 0:
        sample = numeric_cols[:5]
        print(df[sample].describe().loc[['mean', 'std']].round(2))


[운영지표]
      배달매출금액 비율  배달가능여부     월요일_매출_금액     화요일_매출_금액     수요일_매출_금액
mean       9.32    0.29  1.080850e+08  1.189986e+08  1.229868e+08
std       21.95    0.46  2.035607e+08  2.134351e+08  2.273339e+08

[고객지표]
      남성 20대이하 고객 비중  남성 30대 고객 비중  남성 40대 고객 비중  남성 50대 고객 비중  남성 60대이상 고객 비중
mean           11.42         13.30         10.06         11.09            9.07
std            10.04          8.89          8.20         10.26           11.97

[경쟁밀집지표]
      동일 업종 매출금액 비율  동일 업종 매출건수 비율  동일 업종 내 매출 순위 비율  동일 상권 내 매출 순위 비율  \
mean         134.66         152.33             31.14             24.05   
std          168.08         226.85             25.23             22.81   

      동일 업종 내 해지 가맹점 비중  
mean              16.19  
std                3.87  

[접근성지표]
      1km내_버스정류장수  1km내_총버스승하차  1km내_지하철역수  1km내_총지하철승하차  500m내_버스정류장수
mean       301.72  36116003.61        3.61    4671055.19         84.46
std         94.34  16751124.72        1.20    2472843.83         36.53

[경제환경지표]
      

## 4. 결과 저장

In [4]:
# JSON 저장
output_path = '/Users/yeong-gwang/Documents/배움 오전 1.38.42/외부/공모전/빅콘테스트/Project/work/ver3_/1012/result/지표군_분류.json'
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(categories, f, ensure_ascii=False, indent=2)
print(f'저장 완료: {output_path}')

저장 완료: /Users/yeong-gwang/Documents/배움 오전 1.38.42/외부/공모전/빅콘테스트/Project/work/ver3_/1012/result/지표군_분류.json


## 핵심 인사이트

### 지표군 분류 결과
1. **운영지표:** 가맹점의 직접적 운영 성과 (매출, 배달 등)
2. **고객지표:** 고객 특성 및 행동 패턴
3. **경쟁밀집지표:** 시장 경쟁 강도
4. **접근성지표:** 물리적 접근 용이성
5. **경제환경지표:** 외부 거시경제 여건

### 다음 단계
- 5개 지표군 기반 클러스터링 분석
- 지표군별 고위험/정상 차이 검증
- 정책 방향성 도출

---